In [1]:
import pandas as pd
import numpy as np
import sklearn
import scipy
import plotly.express as px

In [2]:
import numpy as np
from scipy.optimize import linprog

# Define horses and their winning odds
horses = ['Horse' + str(i+1) for i in range(10)]
odds = [8, 10, 5, 15, 12, 15, 20, 12, 9/5, 7/2]

# Convert odds to decimal payouts
decimal_payouts = [odd + 1 for odd in odds]

# Calculate implied probabilities
probabilities = [1 / payout for payout in decimal_payouts]

# Budget
budget = 100

# Calculate EV for Win bet type (assuming we focus on Win bets for this example)
ev_win = [(prob * payout - (1 - prob)) for prob, payout in zip(probabilities, decimal_payouts)]

# Objective function coefficients (negative because linprog does minimization)
c = -np.array(ev_win)

# Constraints
A_eq = [1] * len(c)
b_eq = [budget]

# Bounds
x_bounds = [(0, None) for _ in range(len(c))]

# Optimization
res = linprog(c, A_eq=[A_eq], b_eq=b_eq, bounds=x_bounds, method='highs')

# Results
bets = res.x
total_ev = -res.fun

# Output results
print("Optimal Bets:", bets)
print("Total Expected Value:", total_ev)


Optimal Bets: [  0.   0.   0.   0.   0.   0.   0.   0. 100.   0.]
Total Expected Value: 35.71428571428572


In [3]:
import numpy as np
from scipy.optimize import linprog

# Define horses and their winning odds
horses = ['Horse' + str(i+1) for i in range(10)]
odds = [8, 10, 5, 15, 12, 15, 20, 12, 9/5, 7/2]

# Convert odds to decimal payouts
decimal_payouts = [odd + 1 for odd in odds]

# Calculate implied probabilities
probabilities = [1 / payout for payout in decimal_payouts]

# Budget
budget = 100
max_bet_per_horse = budget / 3  # Example: limiting the bet to a third of the budget per horse

# Calculate EV for Win bet type
ev_win = [(prob * payout - (1 - prob)) for prob, payout in zip(probabilities, decimal_payouts)]

# Objective function coefficients (negative because linprog does minimization)
c = -np.array(ev_win)

# Constraints
A_eq = [1] * len(c)
b_eq = [budget]

# Inequality constraints for diversification (max bet per horse)
A_ub = np.identity(len(c))
b_ub = [max_bet_per_horse] * len(c)

# Bounds
x_bounds = [(0, None) for _ in range(len(c))]

# Optimization
res = linprog(c, A_eq=[A_eq], b_eq=b_eq, A_ub=A_ub, b_ub=b_ub, bounds=x_bounds, method='highs')

# Results
bets = res.x
total_ev = -res.fun

# Output results
print("Optimal Bets:", bets)
print("Total Expected Value:", total_ev)


Optimal Bets: [-1.42108547e-14  0.00000000e+00  3.33333333e+01  0.00000000e+00
  0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
  3.33333333e+01  3.33333333e+01]
Total Expected Value: 24.86772486772487


In [4]:
import numpy as np
from scipy.optimize import linprog

# Define horses and their winning odds
horses = ['Seize the Grey', 'Resilience', 'Mystik Dan', 'The Wine Steward', 'Antiquarian',
          'Dornoch', 'Protective', 'Honor Marie', 'Sierra Leone', 'Mindframe']
odds = [8, 10, 5, 15, 12, 15, 20, 12, 9/5, 7/2]

# Convert odds to decimal payouts
decimal_payouts = [odd + 1 for odd in odds]

# Calculate implied probabilities
probabilities = [1 / payout for payout in decimal_payouts]

# Budget range
min_budget = 100
max_budget = 500
max_bet_per_horse = max_budget / 4  # Example: limiting the bet to a quarter of the maximum budget per horse

# Calculate EV for Win bet type
ev_win = [(prob * payout - (1 - prob)) for prob, payout in zip(probabilities, decimal_payouts)]

# Objective function coefficients (negative because linprog does minimization)
c = -np.array(ev_win)

# Constraints
A_eq = np.ones((1, len(c)))
b_eq_min = [min_budget]
b_eq_max = [max_budget]

# Inequality constraints for diversification (max bet per horse)
A_ub = np.identity(len(c))
b_ub = [max_bet_per_horse] * len(c)

# Combine constraints
A = np.vstack([A_eq, A_ub])
b = np.hstack([b_eq_max, b_ub])

# Bounds
x_bounds = [(0, None) for _ in range(len(c))]

# Optimization
res = linprog(c, A_eq=A_eq, b_eq=b_eq_min, A_ub=A, b_ub=b, bounds=x_bounds, method='highs')

# Results
bets = res.x
total_ev = -res.fun

# Output results
print("Optimal Bets:", bets)
print("Total Expected Value:", total_ev)


Optimal Bets: [  0.   0.   0.   0.   0.   0.   0.   0. 100.   0.]
Total Expected Value: 35.71428571428572


In [5]:
import numpy as np
from scipy.optimize import linprog
from scipy.optimize import milp

# Define horses and their winning odds
horses = ['Seize the Grey', 'Resilience', 'Mystik Dan', 'The Wine Steward', 'Antiquarian',
          'Dornoch', 'Protective', 'Honor Marie', 'Sierra Leone', 'Mindframe']
odds = [8, 10, 5, 15, 12, 15, 20, 12, 9/5, 7/2]

# Convert odds to decimal payouts
decimal_payouts = [odd + 1 for odd in odds]

# Calculate implied probabilities
probabilities = [1 / payout for payout in decimal_payouts]

# Budget range
min_budget = 100
max_budget = 500
max_bet_per_horse = max_budget / 4  # Example: limiting the bet to a quarter of the maximum budget per horse

# Calculate EV for Win bet type
ev_win = [(prob * payout - (1 - prob)) for prob, payout in zip(probabilities, decimal_payouts)]

# Objective function coefficients (negative because linprog does minimization)
c = -np.array(ev_win + [0] * len(horses))

# Constraints
A_eq = np.ones((1, len(horses))) # Total bet constraint
b_eq_min = [min_budget]
b_eq_max = [max_budget]

# Inequality constraints for diversification (max bet per horse)
A_ub = np.identity(len(horses))
b_ub = [max_bet_per_horse] * len(horses)

# Binary constraints for ensuring minimum number of bets
bet_indicator = np.eye(len(horses))
bet_min = np.zeros(len(horses))
bet_max = [1] * len(horses)
A_min_bets = np.ones((1, len(horses)))
b_min_bets = [5]  # At least 5 horses must be bet on

# Combine all constraints
A = np.vstack([A_eq, A_ub, A_min_bets])
b = np.hstack([b_eq_max, b_ub, b_min_bets])

# Bounds for bet amounts and binary indicators
x_bounds = [(0, None) for _ in range(len(horses))]

# Integrate binary variables
integrality = np.zeros(len(horses))
integrality[:len(horses)] = 1

# Optimization
res = milp(c, integrality=integrality, bounds=x_bounds, constraints={'A_eq': A_eq, 'b_eq': b_eq_min, 'A_ub': A, 'b_ub': b})

# Results
bets = res.x[:len(horses)]
total_ev = -res.fun

# Output results
print("Optimal Bets:", bets)
print("Total Expected Value:", total_ev)


ValueError: `integrality` must contain integers 0-3 and be broadcastable to `c.shape`.

In [6]:
import numpy as np
from scipy.optimize import linprog

# Define horses and their winning odds
horses = ['Seize the Grey', 'Resilience', 'Mystik Dan', 'The Wine Steward', 'Antiquarian',
          'Dornoch', 'Protective', 'Honor Marie', 'Sierra Leone', 'Mindframe']
odds = [8, 10, 5, 15, 12, 15, 20, 12, 9/5, 7/2]

# Convert odds to decimal payouts
decimal_payouts = [odd + 1 for odd in odds]

# Calculate implied probabilities
probabilities = [1 / payout for payout in decimal_payouts]

# Budget range
min_budget = 100
max_budget = 500
max_bet_per_horse = max_budget / 4  # Example: limiting the bet to a quarter of the maximum budget per horse

# Calculate EV for Win bet type
ev_win = [(prob * payout - (1 - prob)) for prob, payout in zip(probabilities, decimal_payouts)]

# Objective function coefficients (negative because linprog does minimization)
c = -np.array(ev_win)

# Constraints
A_eq = [np.ones(len(c))]
b_eq = [min_budget]

# Inequality constraints for diversification (max bet per horse)
A_ub = np.vstack([np.eye(len(c)), np.ones(len(c))])
b_ub = [max_bet_per_horse] * len(c) + [max_budget]

# Bounds
x_bounds = [(0, None) for _ in range(len(c))]

# Solve the linear programming problem
res = linprog(c, A_eq=A_eq, b_eq=b_eq, A_ub=A_ub, b_ub=b_ub, bounds=x_bounds, method='highs')

# Binary optimization to ensure at least 5 horses are bet on
if res.success:
    initial_bets = res.x
    total_bet = sum(initial_bets)
    if total_bet >= min_budget and total_bet <= max_budget:
        # Sort the horses based on the bet amount
        sorted_bets = sorted(enumerate(initial_bets), key=lambda x: -x[1])
        final_bets = [0] * len(horses)
        for i in range(5):
            index = sorted_bets[i][0]
            final_bets[index] = min(sorted_bets[i][1], max_bet_per_horse)
        remaining_budget = max_budget - sum(final_bets)
        for i in range(5, len(horses)):
            index = sorted_bets[i][0]
            if remaining_budget <= 0:
                break
            bet_amount = min(remaining_budget, sorted_bets[i][1], max_bet_per_horse)
            final_bets[index] = bet_amount
            remaining_budget -= bet_amount

        total_ev = sum(ev_win[i] * final_bets[i] for i in range(len(horses)))
        print("Optimal Bets:", final_bets)
        print("Total Expected Value:", total_ev)
    else:
        print("Initial optimization did not meet the budget constraints.")
else:
    print("Optimization failed.")


Optimal Bets: [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 100.0, 0.0]
Total Expected Value: 35.71428571428572


In [7]:
import numpy as np
from scipy.optimize import linprog

# Define horses and their winning odds
horses = ['Seize the Grey', 'Resilience', 'Mystik Dan', 'The Wine Steward', 'Antiquarian',
          'Dornoch', 'Protective', 'Honor Marie', 'Sierra Leone', 'Mindframe']
odds = [8, 10, 5, 15, 12, 15, 20, 12, 9/5, 7/2]

# Convert odds to decimal payouts
decimal_payouts = [odd + 1 for odd in odds]

# Calculate implied probabilities
probabilities = [1 / payout for payout in decimal_payouts]

# Budget
budget = 100

# Calculate EV for Win bet type
ev_win = [(prob * payout - (1 - prob)) for prob, payout in zip(probabilities, decimal_payouts)]

# Example for Place and Show bets (these are simplifications, real calculation might be more complex)
# For Place, you win if your horse finishes 1st or 2nd
ev_place = [(2 * prob * payout - (1 - 2 * prob)) for prob, payout in zip(probabilities, decimal_payouts)]

# For Show, you win if your horse finishes 1st, 2nd, or 3rd
ev_show = [(3 * prob * payout - (1 - 3 * prob)) for prob, payout in zip(probabilities, decimal_payouts)]

# Combine all expected values
ev_all = ev_win + ev_place + ev_show

# Objective function coefficients (negative because linprog does minimization)
c = -np.array(ev_all)

# Constraints
A_eq = [[1] * len(ev_win) + [0] * len(ev_win) + [0] * len(ev_win),
        [0] * len(ev_win) + [1] * len(ev_win) + [0] * len(ev_win),
        [0] * len(ev_win) + [0] * len(ev_win) + [1] * len(ev_win)]
b_eq = [budget / 3, budget / 3, budget / 3]

# Inequality constraints (no more than budget)
A_ub = np.ones((1, len(c)))
b_ub = [budget]

# Bounds
x_bounds = [(0, None) for _ in range(len(c))]

# Optimization
res = linprog(c, A_eq=A_eq, b_eq=b_eq, A_ub=A_ub, b_ub=b_ub, bounds=x_bounds, method='highs')

# Results
bets = res.x
total_ev = -res.fun

# Output results
print("Optimal Bets:", bets)
print("Total Expected Value:", total_ev)


Optimal Bets: [ 0.          0.          0.          0.          0.          0.
  0.          0.         33.33333333  0.          0.          0.
  0.          0.          0.          0.          0.          0.
 33.33333333  0.          0.          0.          0.          0.
  0.          0.          0.          0.         33.33333333  0.        ]
Total Expected Value: 171.42857142857144


In [8]:
import numpy as np
from scipy.optimize import linprog

# Define horses and their winning odds
horses = ['Seize the Grey', 'Resilience', 'Mystik Dan', 'The Wine Steward', 'Antiquarian',
          'Dornoch', 'Protective', 'Honor Marie', 'Sierra Leone', 'Mindframe']
odds = [8, 10, 5, 15, 12, 15, 20, 12, 9/5, 7/2]

# Convert odds to decimal payouts
decimal_payouts = [odd + 1 for odd in odds]

# Calculate implied probabilities
probabilities = [1 / payout for payout in decimal_payouts]

# Budget
budget = 100

# Calculate EV for Win, Place, and Show bet types
ev_win = [(prob * payout - (1 - prob)) for prob, payout in zip(probabilities, decimal_payouts)]
ev_place = [(2 * prob * payout - (1 - 2 * prob)) for prob, payout in zip(probabilities, decimal_payouts)]
ev_show = [(3 * prob * payout - (1 - 3 * prob)) for prob, payout in zip(probabilities, decimal_payouts)]

# Combine all expected values
ev_all = ev_win + ev_place + ev_show

# Objective function coefficients (negative because linprog does minimization)
c = -np.array(ev_all)

# Constraints
A_eq = [[1] * len(ev_win) + [0] * len(ev_win) + [0] * len(ev_win),
        [0] * len(ev_win) + [1] * len(ev_win) + [0] * len(ev_win),
        [0] * len(ev_win) + [0] * len(ev_win) + [1] * len(ev_win)]
b_eq = [budget / 3, budget / 3, budget / 3]

# Inequality constraints for budget and diversification
A_ub = np.ones((1, len(c)))
b_ub = [budget]

# Additional constraint to limit the amount bet on any single horse to $33
A_ub = np.vstack([A_ub, np.identity(len(c))])
b_ub = np.hstack([b_ub, [33] * len(c)])

# Bounds
x_bounds = [(0, None) for _ in range(len(c))]

# Optimization
res = linprog(c, A_eq=A_eq, b_eq=b_eq, A_ub=A_ub, b_ub=b_ub, bounds=x_bounds, method='highs')

# Results
bets = res.x
total_ev = -res.fun

# Output results
print("Optimal Bets:", bets)
print("Total Expected Value:", total_ev)


Optimal Bets: [ 0.          0.          0.          0.          0.          0.
  0.          0.         33.          0.33333333  0.          0.
  0.          0.          0.          0.          0.          0.
 33.          0.33333333  0.          0.          0.          0.
  0.          0.          0.          0.         33.          0.33333333]
Total Expected Value: 171.15873015873015


In [9]:
def calculate_ev(prob_win, payout, bet_amount):
    return (prob_win * payout * bet_amount) - ((1 - prob_win) * bet_amount)

# Sierra Leone
ev_win_sierra = calculate_ev(0.5556, 2.8, 11.11)
ev_place_sierra = calculate_ev(1.1112, 2.0, 11.11)
ev_show_sierra = calculate_ev(1.6668, 1.67, 11.11)

# Mindframe
ev_win_mind = calculate_ev(0.2857, 4.5, 11.11)
ev_place_mind = calculate_ev(0.5714, 3.5, 11.11)
ev_show_mind = calculate_ev(0.8571, 2.67, 11.11)

# Seize the Grey
ev_win_seize = calculate_ev(0.1111, 10.0, 11.11)
ev_place_seize = calculate_ev(0.2222, 6.0, 11.11)
ev_show_seize = calculate_ev(0.3333, 3.67, 11.11)

# Total EV for Equal Betting
total_ev_equal = (ev_win_sierra + ev_place_sierra + ev_show_sierra +
                  ev_win_mind + ev_place_mind + ev_show_mind +
                  ev_win_seize + ev_place_seize + ev_show_seize)

total_ev_equal



139.06891394

In [10]:
# Proportional Bet Amounts
bet_sierra = 12.59 / 3
bet_mind = 24.48 / 3
bet_seize = 62.93 / 3

# Sierra Leone
ev_win_sierra_p = calculate_ev(0.5556, 2.8, bet_sierra)
ev_place_sierra_p = calculate_ev(1.1112, 2.0, bet_sierra)
ev_show_sierra_p = calculate_ev(1.6668, 1.67, bet_sierra)

# Mindframe
ev_win_mind_p = calculate_ev(0.2857, 4.5, bet_mind)
ev_place_mind_p = calculate_ev(0.5714, 3.5, bet_mind)
ev_show_mind_p = calculate_ev(0.8571, 2.67, bet_mind)

# Seize the Grey
ev_win_seize_p = calculate_ev(0.1111, 10.0, bet_seize)
ev_place_seize_p = calculate_ev(0.2222, 6.0, bet_seize)
ev_show_seize_p = calculate_ev(0.3333, 3.67, bet_seize)

# Total EV for Proportional Betting
total_ev_proportional = (ev_win_sierra_p + ev_place_sierra_p + ev_show_sierra_p +
                         ev_win_mind_p + ev_place_mind_p + ev_show_mind_p +
                         ev_win_seize_p + ev_place_seize_p + ev_show_seize_p)

total_ev_proportional


91.91188027666666

In [11]:
# Define horses and their winning odds
horses = ['Seize the Grey', 'Resilience', 'Mystik Dan', 'The Wine Steward', 'Antiquarian',
          'Dornoch', 'Protective', 'Honor Marie', 'Sierra Leone', 'Mindframe']
odds = [9.0, 11.0, 6.0, 16.0, 13.0, 16.0, 21.0, 13.0, 1.8, 4.5]

# Calculate inverse of the odds
inverse_odds = [1/odd for odd in odds]

# Normalize inverse odds to sum to 1
total_inverse_odds = sum(inverse_odds)
normalized_inverse_odds = [x/total_inverse_odds for x in inverse_odds]

# Budget
budget = 100

# Allocate budget inversely proportional to the odds
allocated_bets = [budget * x for x in normalized_inverse_odds]

# Split each allocated bet equally among Win, Place, and Show
win_bets = [x / 3 for x in allocated_bets]
place_bets = [x / 3 for x in allocated_bets]
show_bets = [x / 3 for x in allocated_bets]

# Combine all bets into one list for easier handling
all_bets = win_bets + place_bets + show_bets

# Print allocated bets
print("Allocated Bets (Win, Place, Show for each horse):")
for i in range(len(horses)):
    print(f"{horses[i]} - Win: {win_bets[i]:.2f}, Place: {place_bets[i]:.2f}, Show: {show_bets[i]:.2f}")

# Print combined bets
print("All Bets:", all_bets)


Allocated Bets (Win, Place, Show for each horse):
Seize the Grey - Win: 2.51, Place: 2.51, Show: 2.51
Resilience - Win: 2.06, Place: 2.06, Show: 2.06
Mystik Dan - Win: 3.77, Place: 3.77, Show: 3.77
The Wine Steward - Win: 1.41, Place: 1.41, Show: 1.41
Antiquarian - Win: 1.74, Place: 1.74, Show: 1.74
Dornoch - Win: 1.41, Place: 1.41, Show: 1.41
Protective - Win: 1.08, Place: 1.08, Show: 1.08
Honor Marie - Win: 1.74, Place: 1.74, Show: 1.74
Sierra Leone - Win: 12.57, Place: 12.57, Show: 12.57
Mindframe - Win: 5.03, Place: 5.03, Show: 5.03
All Bets: [2.5145146653855455, 2.0573301807699917, 3.771771998078318, 1.4144144992793695, 1.740817845266916, 1.4144144992793695, 1.077649142308091, 1.740817845266916, 12.572573326927726, 5.029029330771091, 2.5145146653855455, 2.0573301807699917, 3.771771998078318, 1.4144144992793695, 1.740817845266916, 1.4144144992793695, 1.077649142308091, 1.740817845266916, 12.572573326927726, 5.029029330771091, 2.5145146653855455, 2.0573301807699917, 3.77177199807831

In [12]:
# Calculate expected value for each bet
def calculate_ev(prob_win, payout, bet_amount):
    return (prob_win * payout * bet_amount) - ((1 - prob_win) * bet_amount)

# Calculate expected returns and total EV
ev_win = []
ev_place = []
ev_show = []
for i in range(len(horses)):
    win_prob = 1 / odds[i]
    place_prob = min(2 * win_prob, 1)  # Capping at 1 for simplification
    show_prob = min(3 * win_prob, 1)  # Capping at 1 for simplification
    
    ev_win.append(calculate_ev(win_prob, odds[i] + 1, win_bets[i]))
    ev_place.append(calculate_ev(place_prob, 2, place_bets[i]))  # Assuming place payout as 2
    ev_show.append(calculate_ev(show_prob, 1.67, show_bets[i]))  # Assuming show payout as 1.67

# Total Expected Value
total_ev = sum(ev_win) + sum(ev_place) + sum(ev_show)

# Print Expected Value
print("Total Expected Value:", total_ev)


Total Expected Value: 61.95320098122845
